In [1]:
# intall azure cog. search package if not done yet
#%pip install azure-search-documents

In [2]:
# get key
import pandas as pd

keys = pd.read_csv("keys/keys_hezhang.csv", index_col=0)
openai_key = keys.loc["openai", "key_value"]
speech_key = keys.loc["search", "key_value"]

In [3]:
import openai
openai.api_type = "azure"
openai.api_base = "https://explore-openai.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = openai_key

model = "text-davinci-003"
text_search_doc_embedding_model = "curie-search-doc"
TEXT_SEARCH_QUERY_EMBEDDING_ENGINE = "curie-search-query" 

In [4]:
import numpy as np
import pandas as pd

from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from openai.embeddings_utils import get_embedding, cosine_similarity

search_endpoint = "https://cnndailymail-cog-search.search.windows.net"
search_index = "cnndailymail-index-v1"
search_key = speech_key

## 1. Load Data 

In [5]:
# read the CNN dailymail dataset
df = pd.read_csv("data/cnn_dailymail_data.csv") #path to CNN daily mail dataset
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11490 entries, 0 to 11489
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          11490 non-null  object
 1   article     11490 non-null  object
 2   highlights  11490 non-null  object
dtypes: object(3)
memory usage: 269.4+ KB


,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


## 2. Search Documents Using Azure Cognitive Search 

In [6]:
# search some key words
search_text = "Clinton Democratic Nomination" # 希拉里·克林顿提名民主党候选人
search_client = SearchClient(search_endpoint, search_index, AzureKeyCredential(search_key))
search_results = search_client.search(search_text=search_text, include_total_count=True)

# show top search results
top_n = 5
top_search_results = []
for index, result in enumerate(list(search_results)[:top_n]):
    print("{}: {}\n".format(index+1, result["highlights"]))
    top_search_results.append(result["highlights"])
    
join_top_search_results = " ".join(top_search_results)

1: Laurene Jobs praised former First Lady Hillary Clinton .
Steve Jobs' widow claimed the Democratic front-runner is 'revolutionary'
Mrs Clinton has announced her candidacy and began campaigning in Iowa .
She described Mrs Clinton as 'America's greatest modern creation'

2: Hillary Clinton developed a close relationship with the financial world as a New York senator .
Clinton's allies there are eager to galvanize a broad network of potential donors .
Her coziness with Wall Street irritates liberal activists, who are a growing influence in the Democratic Party .

3: 'Republicans seem to be talking only about me,' Hillary told reporters in New Hampshire .
That's true: A forthcoming book claims she traded official favors at the State Department for speaking fees and donations to her foundation .
'Hopefully we'll get on to the issues,' she said .
Clinton has yet to publicly articulate her own platform of issues and policy goals as a presidential candidate .

4: Bill de Blasio 'wants to be 

## 3. Summarize Text 

In [7]:
# create a prompt for text analytics
prompt = f"""Text: 

             ###
             {join_top_search_results}
             ###
             
             Tasks:
             1. Summarize the above news articles into 3 short sentences.
             2. Determine whether the articles are Positive or Negative.
             3. Extract top 3 key entities that I need to pay attention to.
             
             Answer the above 3 tasks, one at a time.
          """

# openai results
response=openai.Completion.create(
    engine=model,
    prompt=prompt,
    temperature=0,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5)
print(response["choices"][0]["text"].replace("\n", ""))

1. Laurene Jobs praised Hillary Clinton, who has announced her candidacy and began campaigning in Iowa; Bill de Blasio is allegedly backed by Working Families Party to be a more liberal alternative to Clinton; Timothy Stanley believes Democrats need bold new ideas instead of just Hillary Clinton.2. The articles are generally Positive as they focus on the accomplishments of Hillary Clinton and the potential for a liberal primary challenge.3. Key entities: Hillary Clinton, Bill de Blasio, Working Families Party.


In [8]:
# create a prompt for text analytics
prompt = f"""Text: 

             ###
             {join_top_search_results}
             ###
             
             Tasks:
             1. Summarize the above news articles into 3 short sentences.
             2. Determine whether the articles are Positive or Negative.
             3. Extract top 3 key entities that I need to pay attention to.
             
             Answer the above 3 tasks, one at a time, in simplified Chinese language.
          """

# openai results
response=openai.Completion.create(
    engine=model,
    prompt=prompt,
    temperature=0,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5)
print(response["choices"][0]["text"].replace("\n", ""))

1. 劳伦·乔布斯赞扬前第一夫人希拉里·克林顿，克林顿宣布参加总统竞选并在艾奥瓦州开始竞选，比尔·德·布拉西奥试图成为希拉里的一个更加自由的对手。2. 文章整体上是正面的。3. 主要实体有：劳伦·乔布斯、希拉里·克林顿、比尔·德·布拉西奥和工人家庭党。
